In [7]:
import numpy as np
import pandas as pd
import gc
#import lightgbm as lgb
import warnings
import gc
from sklearn.preprocessing import StandardScaler
import matplotlib as  mpl
from matplotlib  import pyplot as plt
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False
from datetime import timedelta
from datetime import datetime
warnings.filterwarnings('ignore')

In [8]:
import os
os.chdir('C:\\Users\\linxiong.ruan\\Desktop')

In [9]:
%%time
dataset = pd.read_csv('dataset.csv', index_col=0)
dbd_faultcode = pd.read_csv('dbd_faultcode.csv', encoding='GBK' )
train_data = dataset.merge(dbd_faultcode,how = 'left',on = ["SPN","FMI"])
up_data = train_data
up_data.COLLECTTIME = pd.to_datetime(up_data.COLLECTTIME)
up_data.sort_values(by='COLLECTTIME', inplace=True)
# 删除明显异常的行
drop_rows = [0, 1, 2, 3]
drop_rows.extend([x for x in range(1875076, 1875088)])
up_data.drop(drop_rows, axis=0, inplace=True)
# 默认填充为0
up_data["FLAG"] = 0
# 筛选出故障编码不为空的UNIQUENO(一个UNIQUENO可假设为一台机器)
prob_uniqs = up_data[up_data.CODE.notnull()].UNIQUENO.unique()

Wall time: 33.3 s


In [10]:
%%time
# 出现故障的时间段进行标记
'''
minutes = 300
'''
# 设定优先级，对于少数种类的故障，优先标记
code_prev = {
    1206: 0,
    1141: 1,
    1182: 2,
    1168: -1,
    1239: -1,
    1168: -1,
    1211: -1,
    1216: -1,
    1209: -1,
    1145: -1,
    1341: -1,
    1246: -1,
    1215: -1,
    1122: -1
}

delta = timedelta(minutes = 300)
left_delta = delta
right_delta = delta
# 从有故障的设备列表中取出一台设备
for dev_id in prob_uniqs:
    # 取出这台设备的全部数据
    cdf = up_data[up_data.UNIQUENO == dev_id].copy()
    cdf['index'] = cdf.index
    cdf.reset_index(inplace=True)
    # 取出这台设备故障码不为空的数据
    notna_cdf = cdf[cdf.CODE.notna()]
#     print(datetime.now(), dev_id, cdf.shape[0], notna_cdf.shape[0])
    cdf_len = cdf.shape[0]
    # 在这台设备数据中遍历不为空的位置
    for i in notna_cdf.index:
        dt = cdf.at[i, 'COLLECTTIME']
        code = cdf.at[i, 'CODE']
        # 在前后时间间隔内打标签
        for y in range(i, -1, -1):
            y_code = up_data.at[cdf.at[y, 'index'], 'FLAG']
            if y_code > 0 and code_prev[y_code] >= code_prev[code]:
                break
            elif cdf.at[y, 'COLLECTTIME'] >= dt - left_delta:
                up_data.at[cdf.at[y, 'index'], 'FLAG'] = code
            else:
                break
        
        for y in range(i + 1, cdf_len, 1):
            y_code = up_data.at[cdf.at[y, 'index'], 'FLAG']
            if y_code > 0 and code_prev[y_code] >= code_prev[code]:
                break
            elif cdf.at[y, 'COLLECTTIME'] <= dt + right_delta:
                up_data.at[cdf.at[y, 'index'], 'FLAG'] = code
            else:
                break
print('done')

done
Wall time: 33.7 s


In [11]:
%%time
#columns_filter,util_cols由人工提供
columns_filter = ['CD_VP_UNIQUENO', 'AREA', 'ROAD', 'LONGITUDE', 'LATITUDE', 'SPEED',
       'DIRECTION', 'ALTITUDE', 'ACCSTATE', 'POWERDOWNSTATE', 'LOCATIONSTATE',
       'WORKHOURS', 'ACCRUNTIMES', 'ACCWORKTIMES', 'PRESSURE',
       'WATERTEMPERATURE', 'OILLOCATION', 'OILTEMPERATURE', 'OILPRESSURE',
       'ROUNDSPEED', 'ACCEXCCOUNT', 'GSMSIGNAL', 'WORKDETAILID',
       'VEHICLESTATUS', 'TERMINALSTATUS', 'SATELLITE', 'TERMINALEL_OUTSIDE',
       'TERMINALEL_INSIDE', 'INSIDE_TEMPERATURE', 'INSIDE_HUMIDITY',
       'MILENUMBER', 'OILVALUE1', 'OILVALUE2', 'MIXSTATE', 'QUSHOILTIMES',
       'ENVTEMPERATURE', 'PULSESIGNS', 'ENVHUMIDITY', 'OILPERCENT1',
       'ELCANDVOL', 'WIRELESSTEMP1', 'WIRELESSTEMP2', 'WIRELESSTEMP3',
       'WIRELESSTEMP4', 'SIMNO', 'OPTOILLEVEL', 'ENGINELOADSTATE', 'CMDTYPE',
       'ISANALYSED', 'VERSIONNUM', 'LOCK_INSTRUMENT', 'ALARM_0010',
       'ALARM_0003', 'ALARM_0008', 'ALARM_0005', 'ALARM_0006', 'ALARM_0007',
       'ALARM_0009', 'LOWVOLTAGESTATUS', 'ALARM_0002', 'ALARM_0004',
       'ALARM_0001', 'THROTTLECURRENT', 'CURRENTSPEEDLOADPERCENTAGE',
       'ENGINETORQUEPERCENT', 'ROTATIONALSPEED', 'ENGINERUNTIMES',
       'COOLANTTEMPERATURE', 'ENGINEOILTEMPERATURE', 'INTAKETEMPERATURE',
       'TEMPERATURE', 'LUBRICATINGOILPRESSURE', 'COOLANTLEVEL',
       'INTAKEMANIFOLDTEMPERATURE', 'FUELUSETOTAL', 'OILCONSUMPTIONRATE',
       'CURRENTFAULTCOUNT1', 'CURRENTFAULTCOUNT2', 'CURRENTFAULTCOUNT3',
       'CURRENTFAULTCOUNT4', 'CURRENTFAULTCOUNT5', 'CURRENTFAULTCOUNT6','FLAG','UNIQUENO','COLLECTTIME']
util_cols = ['COLLECTTIME','FLAG','UNIQUENO']

# 筛选出这四类故障的数据
x_train = up_data[up_data.FLAG.isin({0,1206,1141,1182})]
x_data = x_train[columns_filter]
y_train = x_data['FLAG']
y_train.replace(1206,1,inplace=True)
y_train.replace(1141,2,inplace=True)
y_train.replace(1182,3,inplace=True)
x_data['FLAG'] = y_train.values 
x_data.index = pd.to_datetime(x_data['COLLECTTIME'])

Wall time: 1.5 s


In [12]:
train_x_cols = ['UNIQUENO','CD_VP_UNIQUENO', 'AREA', 'ROAD', 'LONGITUDE', 'LATITUDE', 'SPEED',
       'DIRECTION', 'ALTITUDE', 'ACCSTATE', 'POWERDOWNSTATE', 'LOCATIONSTATE',
       'WORKHOURS', 'ACCRUNTIMES', 'ACCWORKTIMES', 'PRESSURE',
       'WATERTEMPERATURE', 'OILLOCATION', 'OILTEMPERATURE', 'OILPRESSURE',
       'ROUNDSPEED', 'ACCEXCCOUNT', 'GSMSIGNAL', 'WORKDETAILID',
       'VEHICLESTATUS', 'TERMINALSTATUS', 'SATELLITE', 'TERMINALEL_OUTSIDE',
       'TERMINALEL_INSIDE', 'INSIDE_TEMPERATURE', 'INSIDE_HUMIDITY',
       'MILENUMBER', 'OILVALUE1', 'OILVALUE2', 'MIXSTATE', 'QUSHOILTIMES',
       'ENVTEMPERATURE', 'PULSESIGNS', 'ENVHUMIDITY', 'OILPERCENT1',
       'ELCANDVOL', 'WIRELESSTEMP1', 'WIRELESSTEMP2', 'WIRELESSTEMP3',
       'WIRELESSTEMP4', 'SIMNO', 'OPTOILLEVEL', 'ENGINELOADSTATE', 'CMDTYPE',
       'ISANALYSED', 'VERSIONNUM', 'LOCK_INSTRUMENT', 'ALARM_0010',
       'ALARM_0003', 'ALARM_0008', 'ALARM_0005', 'ALARM_0006', 'ALARM_0007',
       'ALARM_0009', 'LOWVOLTAGESTATUS', 'ALARM_0002', 'ALARM_0004',
       'ALARM_0001', 'THROTTLECURRENT', 'CURRENTSPEEDLOADPERCENTAGE',
       'ENGINETORQUEPERCENT', 'ROTATIONALSPEED', 'ENGINERUNTIMES',
       'COOLANTTEMPERATURE', 'ENGINEOILTEMPERATURE', 'INTAKETEMPERATURE',
       'TEMPERATURE', 'LUBRICATINGOILPRESSURE', 'COOLANTLEVEL',
       'INTAKEMANIFOLDTEMPERATURE', 'FUELUSETOTAL', 'OILCONSUMPTIONRATE',
       'CURRENTFAULTCOUNT1', 'CURRENTFAULTCOUNT2', 'CURRENTFAULTCOUNT3',
       'CURRENTFAULTCOUNT4', 'CURRENTFAULTCOUNT5', 'CURRENTFAULTCOUNT6']
train_y_col = ['UNIQUENO','FLAG']
train_ax = x_data[train_x_cols]
train_ay = x_data[train_y_col]

In [13]:
"""删方差为0的列"""
def drop_wubianhua(data):
    wubianhualie=data.columns[np.std(data,axis=0)==0]
    return data.drop(wubianhualie,axis=1)


In [14]:
"""
计算fisher得分,筛选得分高于平均值的变量 。该步骤需要提前标准化
"""
def fisher(data,y):  ###data是包含健康，故障数据,不包含y，y是健康与否
    def fisher(x,y):
        guzhang=x[y.FLAG==1]
        jiankang=x[y.FLAG==0]
        return np.square(np.mean(guzhang)-np.mean(jiankang))/np.sqrt((np.var(guzhang)+np.var(jiankang)))
    f=np.zeros(data.shape[1])
    tem=0
    for i in range(data.shape[1]):
        f[tem]=fisher(data.iloc[:,i:(i+1)],y)
        tem=tem+1
    return f


In [15]:
"""
相关系数大于0.9的取fisher得分高的
"""
cor_value=0.9
def cor_fisher(data,y):
    def fisher(x,y):
        guzhang=x[y.FLAG==1]
        jiankang=x[y.FLAG==0]
        return np.square(np.mean(guzhang)-np.mean(jiankang))/np.sqrt((np.var(guzhang)+np.var(jiankang)))

    cor=data.corr()

    cor=np.triu(cor,1)

    loc=np.where(cor>cor_value)

    loc=pd.DataFrame({"n":loc[0],"p":loc[1]})

    tem=[]
    for i in range(loc.shape[0]):
        row=np.array(loc.iloc[i:i+1,:])
        row=data.columns[row[0]]
        d1=data[row[0]]
        d2=data[row[1]]
        f1=fisher(d1,y)
        f2=fisher(d2,y)
        tem.append(row[(f1>f2)*1])

    return data.drop(tem,axis=1)



In [16]:

def CDF_out(data,newdata,pvalue=0.95):   ####该处data是一个peer的所有数据
    newdata=np.array(newdata)
    n=newdata.shape[0]
    p=newdata.shape[1]
    re=np.zeros((n,p))
    def CDF(data):
            return np.percentile(data,1-pvalue)
            
    for j in range(p):
        inter=CDF(data.iloc[:,j])
        for i in range(n):
            re[i,j]=((newdata[i,j]>inter))
    return re.sum(axis=1)

#print(count_all)
def score(score): 
    k=10/(np.min(score)-np.median(score))
    return 90+k*(score-np.median(score))

def pinghua(score,alpha=0.5):
    for i in range(1,len(score)):
            score[i]=(1-alpha)*score[i-1]+alpha*score[i]
    return score

def standard_scale(x):
    return (np.mean(x,axis=0),np.std(x,axis=0))

def s_type(data):
    sk=data.skew(axis=0)
    if(sum(sk<-2)>0):
        data[data.columns[sk<-2]]=-data[data.columns[sk<-2]]
    if(sum((sk<=2)*(sk>=-2))>0):
        data[data.columns[(sk<=2)*(sk>=-2)]]=abs(data[data.columns[(sk<=2)*(sk>=-2)]])
    return data

class score_cdf(object):
    def __init__(self,train_ax,train_ay):
        self.train_ax=train_ax
        self.train_ay=train_ay
    def datahandling(self,pr=False):
        self.ID=self.train_ax.UNIQUENO
        self.train_ax=self.train_ax.drop(["UNIQUENO"],axis=1)
        self.train_ax.fillna(0,inplace=True)
        self.train_ax=drop_wubianhua(self.train_ax)
        (self.mean,self.std) = standard_scale(self.train_ax)
        self.train_ax=(self.train_ax-self.mean)/self.std
        self.sk=self.train_ax.skew(axis=0)
        if(sum(self.sk<-2)>0):
            self.train_ax[self.train_ax.columns[self.sk<-2]]=-self.train_ax[self.train_ax.columns[self.sk<-2]]
        if(sum((self.sk<=2)*(self.sk>=-2))>0):
            self.train_ax[self.train_ax.columns[(self.sk<=2)*(self.sk>=-2)]]=abs(self.train_ax[self.train_ax.columns[(self.sk<=2)*(self.sk>=-2)]])
        f=fisher(self.train_ax,self.train_ay)
        self.train_ax=self.train_ax[self.train_ax.columns[f>np.mean(f)]]
        self.train_ax=cor_fisher(self.train_ax,self.train_ay)
        (self.mean,self.std,self.sk)=(self.mean[self.train_ax.columns],self.std[self.train_ax.columns],self.sk[self.train_ax.columns])
        if pr:
            print("选出的变量：",self.train_ax.columns)
    def Inter(self,pvalue=0.95):
        def CDF(data,pvalue):
            return np.percentile(data,1-pvalue)
        Int=np.zeros(self.train_ax.shape[1])
        k=0
        for i in self.train_ax.columns:
            Int[k]=CDF(self.train_ax[i],pvalue)
            k=k+1
        self.Int=Int
    def newdatahand(self,newdata):
        newdata=pd.DataFrame(newdata).T
        newd=newdata[self.train_ax.columns]
        #print(newd)
        newd.fillna(0,inplace=True)
        #print(newd)
        self.newdata=pd.DataFrame(np.array(newd-self.mean)/np.array(self.std),columns=newd.columns)
        #print(newd-self.mean)
        if(sum(self.sk<-2)>0):
            self.newdata[self.newdata.columns[self.sk<-2]]=-self.newdata[self.newdata.columns[self.sk<-2]]
        if(sum((self.sk<=2)*(self.sk>=-2))>0):
            self.newdata[self.newdata.columns[(self.sk<=2)*(self.sk>=-2)]]=abs(self.newdata[self.newdata.columns[(self.sk<=2)*(self.sk>=-2)]])
        #print(self.newdata)
    '''
    def lishidefen(self):
        re=np.zeros(self.train_ax.shape[0])
        for i in range(self.train_ax.shape[0]):
            re[i]=sum(self.train_ax.iloc[i,:]>self.Int)
        self.min=np.min(re)
        self.med=np.median(re)
    '''
    def sc(self,health=True):
        if health:
            self.train_ax = self.train_ax[self.train_ay.FLAG==0]
        count_all=sum(np.array(self.newdata)[0]>self.Int)
        #k=10/(self.min-self.med)
        #score=90+k*(count_all-self.med)
        k=50/(0-7)
        score=100+k*(count_all)
        return score
     

In [57]:
# 1.根据设备的UNIQUENO取数据，做数据处理
'''
201706130830038670
201707201730009538
201801261730005948
'''
'''
UNIQUENO = 201706130830038670
#train_ax_health = train_ax[train_ay.FLAG==0]
'''

In [17]:
y=score_cdf(train_ax,train_ay)
y.datahandling(pr=True)

选出的变量： Index(['LATITUDE', 'LOCATIONSTATE', 'ACCRUNTIMES', 'GSMSIGNAL', 'SATELLITE',
       'CMDTYPE', 'CURRENTFAULTCOUNT1'],
      dtype='object')


In [18]:
y.Inter()

In [19]:
y.newdatahand(train_ax.iloc[1,:])
y.sc()

78.57142857142857

In [20]:
import time
def t(d):
    timeStamp=np.zeros(d.shape[0])
    for i in range(d.shape[0]):
        timeArray = time.strptime(str(d[i]), "%Y-%m-%d %H:%M:%S")
        timeStamp[i] = (int(time.mktime(timeArray))//86400)
    return timeStamp
ID_all=np.unique(train_ax.UNIQUENO)
day=t(train_ax.index)

In [21]:
"""最近10天内所有数据"""
nearday=max(day)
passbyday=max(day)-10

newdata=train_ax[(day<nearday)*(day>passbyday)]

In [ ]:
"""最近10天的所有得分
re=pd.DataFrame(
for i in range(newdata.shape[0]):
    y.newdatahand(newdata.iloc[i,])
    y.sc()
"""

In [22]:
SCORE=pd.DataFrame(index=np.unique(day[(day<nearday)*(day>passbyday)]),columns=np.unique(train_ax.UNIQUENO))

In [23]:
SCORE.index

Float64Index([17642.0, 17643.0, 17644.0, 17645.0, 17646.0, 17647.0, 17648.0,
              17649.0, 17650.0],
             dtype='float64')

In [24]:
i=17650.0
j=201405151234131891
train_ax["xuanze"]=((day==i)*(train_ax.UNIQUENO==j))

In [ ]:
data_oneday_oneid=pd.DataFrame(train_ax[train_ax.xuanze==True].drop(['xuanze'],axis=1),columns=train_ax.columns)
        #data_oneday_oneid=pd.DataFrame(train_ax[(day==i)*(train_ax.UNIQUENO==j),:],columns=train_ax.columns)
oneday_oneid_score=np.zeros(data_oneday_oneid.shape[0])
for k in range(data_oneday_oneid.shape[0]):
            y.newdatahand(data_oneday_oneid.iloc[k,:])
            oneday_oneid_score[k]=y.sc()
SCORE.loc[i,j]=np.max(oneday_oneid_score)

In [ ]:
for i in SCORE.index:
    for j in SCORE.columns:
        train_ax["xuanze"]=((day==i)*(train_ax.UNIQUENO==j))
        data_oneday_oneid=pd.DataFrame(train_ax[train_ax.xuanze==True].drop(['xuanze'],axis=1),columns=train_ax.columns)
        #data_oneday_oneid=pd.DataFrame(train_ax[(day==i)*(train_ax.UNIQUENO==j),:],columns=train_ax.columns)
        oneday_oneid_score=np.zeros(data_oneday_oneid.shape[0])
        for k in range(data_oneday_oneid.shape[0]):
            y.newdatahand(data_oneday_oneid.iloc[k,:])
            oneday_oneid_score[k]=y.sc()
        SCORE.loc[i,j]=np.max(oneday_oneid_score)